In [1]:
%%capture
from tqdm import tqdm_notebook as tqdm

import pandas as pd

import time
t1 = time.time()

import os
import numpy as np

from pandas import ExcelWriter
from pandas import ExcelFile

from datetime import datetime

from matplotlib import cm
from pylab import rcParams
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d
import scipy.interpolate as spi
from scipy.interpolate import splrep, splev
import scipy.cluster.hierarchy as spc

import scipy.io
import math

from sklearn import metrics

In [2]:
#loading the files
file_dir = os.path.join('/Users/digisha/Downloads/Fatigue_project')
file_name = os.path.join(file_dir, 'All_patients_VAS_and_Sleep_diary_data_100819.xlsx')
df = pd.read_excel(file_name, sheetname = "Sheet1")

#df = pd.read_excel(file_with_data, sheetname=sheet_with_data)

df.head()

/Applications/anaconda3/lib/python3.7/site-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


,guid,subject_id,question_id,answer,other,is_checked,timestamp
0,1ec039103ad24b89844763aa92afb1df,63,1600,7,NaN,NaN,09-08-2019 20:51:56
1,a5ece80e89da41bb9d4f3a3baf752a20,63,1601,3,NaN,NaN,09-08-2019 20:52:06
2,d7ccec731d5b4738998985a4126951fd,63,1602,5,NaN,NaN,09-08-2019 20:52:13
3,a8393dc6a0c946f8ba139371bf61ab56,63,1603,1,NaN,NaN,09-08-2019 20:52:20
4,24883197ae844e0da82c60cb75855b08,63,1600,6,NaN,NaN,09-08-2019 21:45:02


In [3]:
#changing datatypes - convering timestamp string to datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])
#drop not required coloumn
df = df.drop(['guid', 'other', 'is_checked'], axis=1)

df = df[df['question_id'].isin([1600, 1601, 1602, 1603])]

#only depression fatigue questionnare considererd - dropping th experiment patient
df = df.loc[df['subject_id'] != 99998].reset_index(drop=True)
df.head()

with tqdm(total= len(df)) as pbar:
    for i in range(0,len(df)):
        if df['subject_id'][i] == 610:
            df['subject_id'][i] = 61
        elif df['subject_id'][i] == 600:
            df['subject_id'][i] = 60
        elif df['subject_id'][i] == 599:
            df['subject_id'][i] = 59
        pbar.update(1)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [4]:
"""k = df.subject_id.unique()
print(np.sort(k))
len(k)"""

'k = df.subject_id.unique()\nprint(np.sort(k))\nlen(k)'

In [5]:
# to check if there is a patient with less than 2 readings- as for interpolating we need atleast 2 reading 
#and will have to drop such patients

#list of patients id
UniqueNames = df.subject_id.unique()

#list of question_id
qid = df.question_id.unique()

#list of patients with less than 2 readings
l1 = []

with tqdm(total= (len(UniqueNames)*len(qid))) as pbar:
    for q in qid:
        for pat in UniqueNames:
            count = 0
            for i in range(0,len(df)):
                if q == df['question_id'][i]:
                    count += 1
            if count <= 1:
                print(pat)
                l1.append(pat)
            
            pbar.update(1)
print(l1)


[]


In [6]:
# date column added to know if the questionnaire is a part of the same day or the next
df['date'] = [d.date() for d in df['timestamp']]

#initiating a datetime obeject as zero time for offset time
datetime_object = datetime.strptime('00:00:00', '%H:%M:%S')

#creating 3 more coloums to get hour, time and day of the data row
df['hour'] = 0.00
df['time'] = 0
df['day'] = 0
df['reading'] = 0

#changing the datatype of anwer column to float
df['time'] = df['time'].astype('float')

initial = 0

df = df.sort_values(by=['subject_id','question_id','timestamp',], ascending=[0, 1, 1]).reset_index(drop=True)

# This may take a while
with tqdm(total= (len(UniqueNames)*len(qid))) as pbar:
    for q in qid:
        for pat in UniqueNames:
            count = 0
            for i in range(0,len(df)):
                if q == df['question_id'][i]:
                    if pat == df['subject_id'][i]:
                        hour = ((df['timestamp'][i] - datetime_object).seconds/(60*60))
                        if count == 0:
                            #hour = ((df['timestamp'][i] - datetime_object).seconds/(60*60))
                            df['time'][i] = hour/24
                            df['day'][i] = 0
                            #df['reading'][i] = 0
                            count+= 1
                            initial = i
                        else:
                            df['day'][i] = (df['date'][i] - df['date'][initial]).days
                            df['time'][i] = df['day'][i] + (hour/24)
                            #(df['timestamp'][i] - df['timestamp'][initial]).days + (hour/24)
                            
                            #df['reading'][i] = df['reading'][i] + 1
                            
                        if i == initial:
                            df['reading'][i] = 0
                        
                        elif df['day'][i] == df['day'][i-1]:
                            df['reading'][i] = df['reading'][i-1]+ 1
                            
                        else:
                            df['reading'][i] = 0
                            
            pbar.update(1)

df['time'] = df['time'].round(3)
#df = df.sort_values(['subject_id', 'time'], ascending=[True, True])
df.head(7)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to 

,subject_id,question_id,answer,timestamp,date,hour,time,day,reading
0,64,1600,0,2019-09-16 17:07:36,2019-09-16,0.0,0.714,0,0
1,64,1600,0,2019-09-16 22:55:10,2019-09-16,0.0,0.955,0,1
2,64,1600,1,2019-09-17 07:26:43,2019-09-17,0.0,1.310,1,0
3,64,1600,0,2019-09-17 11:29:36,2019-09-17,0.0,1.479,1,1
4,64,1600,0,2019-09-17 16:09:52,2019-09-17,0.0,1.674,1,2
5,64,1600,0,2019-09-17 20:18:07,2019-09-17,0.0,1.846,1,3
6,64,1600,0,2019-09-18 09:10:19,2019-09-18,0.0,2.382,2,0


In [7]:
"""mask = (df['subject_id'] >61)
df1 = df.loc[mask].reset_index(drop=True)

mask = (df1['question_id'] < 1602)
df1 = df1.loc[mask].reset_index(drop=True)"""

df1 = df.sort_values(by=['subject_id','question_id','day','reading'], ascending=[1, 1, 0,0]).reset_index(drop=True)

#changing the datatype of answer column to integer
df1['answer'] = df1.answer.astype('int64')

df1.head(5)

,subject_id,question_id,answer,timestamp,date,hour,time,day,reading
0,1,1600,3,2018-06-15 01:07:42,2018-06-15,0.0,15.047,15,0
1,1,1600,3,2018-06-14 20:05:39,2018-06-14,0.0,14.837,14,3
2,1,1600,3,2018-06-14 10:34:48,2018-06-14,0.0,14.441,14,2
3,1,1600,4,2018-06-14 00:55:57,2018-06-14,0.0,14.039,14,1
4,1,1600,4,2018-06-14 00:10:12,2018-06-14,0.0,14.007,14,0


In [8]:
#for pat in [1600,1601,1603,1604]
#slope_qid.append(slope_pat)
slope = []
with tqdm(total= (len(df1.question_id.unique())*len(df1.subject_id.unique()))) as pbar:
    for qid in df1.question_id.unique():
        slope_qid = []
        #total_days = []
        for pat in df1.subject_id.unique():
            slope_pat = []
            slope_pat.append(pat)
            for day in df1.day.unique():
                count = 0
                x1 = 0
                x2 = 0
                y1 = 0
                y2 = 0
                for i in range(0,len(df1)-1):
                    if qid == df1['question_id'][i]:
                        if pat == df1['subject_id'][i]:            
                            if df1['day'][i] == day:
                                #print("k")
                                if count == 0:
                                    #print("l")
                                    #pat_total_day = df1['day'][i]
                                    y2 = df1['answer'][i]
                                    x2 = df1['time'][i]
                                    count +=1
                                if df1['reading'][i] == 0:
                                    #print("p")
                                    y1 = df1['answer'][i]
                                    x1 = df1['time'][i]            
                if x1 == x2:
                    sl = 0
                else:
                    sl = ((y1-y2)/(x1-x2)).round(3)
                #print (day,pat,qid,sl)

                slope_pat.append(sl)

            pbar.update(1)
            #total_days.append(pat_total_day)
            slope_qid.append(slope_pat)
        slope.append(slope_qid)
#slope

In [9]:
total_day = []
with tqdm(total= (len(df1.subject_id.unique()))) as pbar:
    for pat in df1.subject_id.unique():
        day = 0
        for i in range(0,len(df1)-1):
            if df1['question_id'][i] == 1600:
                if df1['subject_id'][i] == pat:
                    if df1['day'][i] >= day:
                        day = df1['day'][i]

        pbar.update(1)
        total_day.append(day)

In [10]:
df1_1600 = pd.DataFrame(slope[0])
df1_1600.rename(columns = {0:'subject_id'}, inplace = True)

In [11]:
df1_1600['mean'] = df1_1600.sum(axis=1)
#df1_1600['mean'] = 0.00
for i in range(0,len(df1_1600)):
    df1_1600['mean'][i] = (df1_1600['mean'][i]/total_day[i]).round(3)

cols = df1_1600.columns.tolist()
cols = cols[0:1] + cols[-1:] + cols[1:-1]
df1_1600 = df1_1600[cols]
df1_1600.to_csv(os.path.join(file_dir, 'output_fatigue_1600.csv'))
df1_1600.head(5)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,subject_id,mean,1,2,3,4,5,6,7,8,...,26,27,28,29,30,31,32,33,34,35
0,1,-0.291,0.000,-1.205,-2.732,-0.000,-1.319,-2.068,-3.233,7.109,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,2,2.473,0.000,-0.000,-3.891,3.003,-1.071,-5.671,7.979,12.153,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,3,-0.531,-0.000,-1.427,-4.732,-0.000,2.491,1.267,-0.000,-2.174,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,4,0.263,1.385,-1.477,-0.000,-1.346,-0.000,1.770,-0.000,-1.862,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,5,-0.058,-1.464,2.519,-5.013,4.380,-0.000,1.041,-0.000,-0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [12]:
df1_1601 = pd.DataFrame(slope[1])
df1_1601.rename(columns = {0:'subject_id'}, inplace = True)

df1_1601['mean'] = df1_1601.sum(axis=1)
for i in range(0,len(df1_1601)):
    df1_1601['mean'][i] = (df1_1601['mean'][i]/total_day[i]).round(3)

cols = df1_1601.columns.tolist()
cols = cols[0:1] + cols[-1:] + cols[1:-1]
df1_1601 = df1_1601[cols]

df1_1601.to_csv(os.path.join(file_dir, 'output_depression_1601.csv'))

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
df1_1602 = pd.DataFrame(slope[2])
df1_1602.rename(columns = {0:'subject_id'}, inplace = True)

df1_1602['mean'] = df1_1602.sum(axis=1)
for i in range(0,len(df1_1602)):
    df1_1602['mean'][i] = (df1_1602['mean'][i]/total_day[i]).round(3)

cols = df1_1602.columns.tolist()
cols = cols[0:1] + cols[-1:] + cols[1:-1]
df1_1602 = df1_1602[cols]

df1_1602.to_csv(os.path.join(file_dir, 'output_anxiety_1602.csv'))

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
df1_1603 = pd.DataFrame(slope[3])
df1_1603.rename(columns = {0:'subject_id'}, inplace = True)

df1_1603['mean'] = df1_1603.sum(axis=1)
for i in range(0,len(df1_1603)):
    df1_1600['mean'][i] = (df1_1603['mean'][i]/total_day[i]).round(3)

cols = df1_1603.columns.tolist()
cols = cols[0:1] + cols[-1:] + cols[1:-1]
df1_1603 = df1_1603[cols]

df1_1603.to_csv(os.path.join(file_dir, 'output_pain_1603.csv'))

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
df1_1603

,subject_id,mean,1,2,3,4,5,6,7,8,...,26,27,28,29,30,31,32,33,34,35
0,1,-0.047,0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,...,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0,0
1,2,0.042,0.000,-0.000,-1.946,-0.000,-2.139,-0.000,1.328,-0.000,...,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0,0
2,3,14.045,-0.000,7.133,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,...,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0,0
3,4,1.028,1.385,-0.000,-3.221,-1.346,-1.383,-1.773,1.140,-1.866,...,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0,0
4,5,4.912,-2.928,-2.516,5.013,-1.462,-0.000,-0.000,-0.000,-0.000,...,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0,0
5,6,22.842,-1.332,-1.063,-1.031,-1.227,-0.000,3.077,-1.107,1.351,...,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0,0
6,7,-1983.610,3.559,-0.000,1.058,0.000,0.000,-0.000,-0.000,-0.000,...,3.252,-0.0,-0.0,-0.0,-0.0,-0.0,3.534,-0.0,0,0
7,8,2.788,-2.732,-0.000,-0.000,-1.351,0.000,-0.000,-0.000,-3.145,...,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0,0
8,9,13.533,-2.000,3.306,1.447,4.213,2.894,-1.464,1.938,-4.360,...,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0,0
9,11,12.001,-4.255,-0.000,1.048,3.472,-0.000,-0.000,1.524,-2.967,...,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0,0


In [16]:
(time.time() - t1)/60

36.254253685474396

In [17]:
#qid(pat(day))